import all libraries
create class 

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.utils import resample

python class for normal logistic regression

In [24]:
class LogisticRegression:
    def __init__(self, weights=None, bias=None, learningRate=0.01, epochs=1000,selectedFeatures=None):
        self.weights = weights
        self.bias = bias
        self.learningRate = learningRate
        self.epochs = epochs
        self.selectedFeatures = selectedFeatures
    

    def loss (self, y, y_hat):
        return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):

        numOfSamples = X.shape[0]
        numOfFeatures = X.shape[1]
        weights = np.zeros(numOfFeatures)
        bias = 0
        learningRate = 0.01
        epochs = 1000
        selectedFeatures = np.random.choice(numOfFeatures, 2, replace=False)
        print(selectedFeatures)

        for i in range(epochs):
            y_hat = self.__sigmoid(np.dot(X, weights) + bias)
            dw = (1 / numOfSamples) * np.dot(X.T, (y_hat - y))
            db = (1 / numOfSamples) * np.sum(y_hat - y)
            weights -= learningRate * dw
            bias -= learningRate * db

            # if i % 100 == 0:
                # print(f'Epoch {i} loss: {self.loss(y, y_hat)}')

        self.weights = weights
        self.bias = bias
        self.selectedFeatures = selectedFeatures

    def predict(self, X):
        y_hat = self.__sigmoid(np.dot(X, self.weights) + self.bias)
        # make sure the output is binary
        y_hat = np.round(y_hat)
        return y_hat    

In [25]:
def dataPreProcessing_creditcard():
    df = pd.read_csv('../Dataset/creditcard.csv')
    numOfAttribuetes = df.shape[1]
    numOfRecords = df.shape[0]
    # print(f'Number of records: {numOfRecords}')
    # print(f'Number of attributes: {numOfAttribuetes}')
    # print(df.columns)
    # print(df.head(10))
    # print(df.describe())
    # print(df.info())

    #1 Checking for missing values  -----> use one less sum to see detailed missing values
    missingValues = df.isnull().sum().sum()
    # print(f'Missing values: {missingValues}')
    #2 Checking for duplicates
    duplicates = df.duplicated().sum()
    # print(f'Duplicates: {duplicates}')
    # duplicateRows = df[df.duplicated()]
    # print(f'Duplicate rows: {duplicateRows}')

    #3 replace missing values with mean ( numerical attributes)
    df.fillna(df.mean(numeric_only=True), inplace=True)

    #4 remove exact duplicates rows, keep the first one
    df.drop_duplicates(inplace=True)
    # print(f'Number of records after removing duplicates: {df.shape[0]}')

    #5 if any target value is missing, remove the record
    df.dropna(subset=['Class'], inplace=True)
    print(f'Number of records after removing missing target values: {df.shape[0]}')

    #6 split the data into features and target
    target_column = 'Class' 
    X = df.drop(target_column, axis=1)
    y = df[target_column]

    # print(X.shape)
    # print(y.shape)
    # print(y.value_counts())
    # print(X.columns)

    #7 display non numerical attributes
    nonNumericalAttributes = X.select_dtypes(include=['object']).columns
    # print(f'Non numerical attributes: {nonNumericalAttributes}')

    # 8 scale the features
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaled_X = scaler.fit_transform(X)

    #9 make x a dataframe again
    X = pd.DataFrame(scaled_X, columns=X.columns)

    return X, y

In [26]:
def dataPreProcessing_iris():
    df = pd.read_csv('../Dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')
    numOfAttribuetes = df.shape[1]
    numOfRecords = df.shape[0]
    print(f'Number of records: {numOfRecords}')
    print(f'Number of attributes: {numOfAttribuetes}')
    print(df.columns)
    print(df.head(10))
    print(df.describe())
    print(df.info())


In [27]:
def test_train_split(X, y, test_size=0.2):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

In [28]:
from sklearn.utils import resample
import numpy as np

def bagging_samples(X_train, y_train):
    # set random seed for reproducibility
    np.random.seed(42)
    # number of samples (rows) in the training set
    n_samples = X_train.shape[0]
    
    # generate 9 bagging samples
    samples = [resample(X_train, y_train, n_samples=n_samples, replace=True) for _ in range(9)]
    
    return samples

In [29]:
def bagging_stacking(data_sets, X_val, y_val):

     # 2 - Train 9 different model with 9 different samples
    models = []
    for i in range(9):
        model = LogisticRegression()
        model.fit(data_sets[i][0], data_sets[i][1])
        models.append(model)

    # 3 -make predictions on the validation set
    y_preds = []
    for model in models:
        y_preds.append(model.predict(X_val))
    y_preds = np.array(y_preds)

    # 4 build a meta model usiiing the predictions from the 9 models
    meta_model = LogisticRegression()
    meta_model.fit(y_preds.T, y_val)

    return models, meta_model

In [30]:
def stackig_predict(models, meta_model, X_test):
    y_preds_test = []
    for model in models:
        y_preds_test.append(model.predict(X_test))
    y_preds_test = np.array(y_preds_test)

    y_pred_meta = meta_model.predict(y_preds_test.T)
    return y_pred_meta

In [31]:
def majority_voting(models, X_test):

    y_preds_test = []
    for model in models:
        y_preds_test.append(model.predict(X_test))
    y_preds_test = np.array(y_preds_test)

    y_pred_majority = np.round(np.mean(y_preds_test, axis=0))
    return y_pred_majority

In [32]:
def LR_star(models, X_test):
# For LR, report average ± stdev for the 9 bagging LR learners
    y_preds_test = []
    for model in models:
        y_preds_test.append(model.predict(X_test))
    y_preds_test = np.array(y_preds_test)
    
    y_pred = np.mean(y_preds_test, axis=0)
    return y_pred

In [33]:
def calculate_metrics(y_test, y_pred):
# Accuracy Sensitivity Specificity Precision F1-score AUROC AUPR
    from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
    accuracy = accuracy_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    aupr = average_precision_score(y_test, y_pred)

    # return as a dictionary
    return {'accuracy': accuracy, 'sensitivity': sensitivity, 'specificity': specificity, 'precision': precision, 'f1': f1, 'auroc': auroc, 'aupr': aupr}

main Function, make changes here to slect different dataset

DataSetLocation:

../Dataset/creditcard.csv
../WA_Fn-UseC_-Telco-Customer-Churn.csv

In [34]:
def main():

# 1 - Data Preprocessing for creditcard dataset
    X,y = dataPreProcessing_creditcard()
    X_train, X_test, y_train, y_test = test_train_split(X, y, test_size=0.2)
    X_train, X_val, y_train, y_val = test_train_split(X_train, y_train, test_size=0.2)
    data_sets = bagging_samples(X_train, y_train)

# 2  - Bagging and Stacking ensemble model preparation
    base_model, meta_model = bagging_stacking(data_sets, X_val, y_val)
# 3 - Predict the test set for stacking_ensemble
    y_pred_meta = stackig_predict(base_model, meta_model, X_test)
    y_pred_voting = majority_voting(base_model, X_test)
    y_pred_LR_star= LR_star(base_model, X_test)

    # make sure the output is binary
    y_pred_meta = np.round(y_pred_meta)
    y_pred_voting = np.round(y_pred_voting)
    y_pred_LR_star = np.round(y_pred_LR_star)
    
# 4 - Calculate matrics of the models
    metrics_meta = calculate_metrics(y_test, y_pred_meta)
    metrics_voting = calculate_metrics(y_test, y_pred_voting)
    metrics_LR_star = calculate_metrics(y_test, y_pred_LR_star)

    print(f'Meta model metrics: {metrics_meta}')
    print(f'Voting model metrics: {metrics_voting}')
    print(f'LR_star model metrics: {metrics_LR_star}')

if __name__ == '__main__':
    main()

Number of records after removing missing target values: 283726
[14  9]
[4 8]
[26  3]
[15 14]
[6 8]
[18  3]
[23  4]
[12  3]
[10  0]
[2 5]
Meta model metrics: {'accuracy': 0.998413985126705, 'sensitivity': np.float64(0.0), 'specificity': np.float64(1.0), 'precision': np.float64(0.0), 'f1': np.float64(0.0), 'auroc': np.float64(0.5), 'aupr': np.float64(0.0015860148732950341)}
Voting model metrics: {'accuracy': 0.9989602791386177, 'sensitivity': np.float64(0.4), 'specificity': np.float64(0.9999117480937588), 'precision': np.float64(0.8780487804878049), 'f1': np.float64(0.549618320610687), 'auroc': np.float64(0.6999558740468794), 'aupr': np.float64(0.352171121119099)}
LR_star model metrics: {'accuracy': 0.9989602791386177, 'sensitivity': np.float64(0.4), 'specificity': np.float64(0.9999117480937588), 'precision': np.float64(0.8780487804878049), 'f1': np.float64(0.549618320610687), 'auroc': np.float64(0.6999558740468794), 'aupr': np.float64(0.352171121119099)}


/home/saimon7/Documents/CSE-472-Machine-Learning/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
